<div style="text-align: center">
    <img style="display: inline-block; height:144px; vertical-align:middle" src="img/shaowei.jpeg" alt="ShaoWei" />
    <img style="display: inline-block; height:144px; vertical-align:middle" src="img/heart.png" alt="❤️" />
    <img style="display: inline-block; height:144px; vertical-align:middle" src="img/julia.png" alt="Julia" />
    <img style="display: inline-block; height:144px; vertical-align:middle" src="img/Meme-like-a-boss.jpg" alt="like a bozz" />
</div>

<div style="font-size:48px; text-align: center">
Walk Like Python; Run Like C
</div>

## Easy to write what I meant

Julia expressions are evaluated on-the-fly, like interpreted languages.

In [1]:
println("Hello, World!")

Hello, World!


String interpolation works.

In [2]:
who_we_are = "MOE ESTL"
println("Hello, we are $(who_we_are[5:end])!")

Hello, we are ESTL!


Functions are JIT compiled. Typing is optional.

In [3]:
function double(x)
    return 2x
end

double (generic function with 1 method)

In [4]:
# Equivalently
double(x) = 2x

double (generic function with 1 method)

Functions can also be overloaded using specialisation.

In [5]:
double(s::Vector) = [s; s]

double (generic function with 2 methods)

In [6]:
@show double(5)
@show double(5.0)
@show double([4, 5, 6])
@show double("dragon")

double(5) = 10
double(5.0) = 10.0
double([4, 5, 6]) = [4, 5, 6, 4, 5, 6]


LoadError: [91mMethodError: no method matching *(::Int64, ::String)[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *(::Real, [91m::Complex{Bool}[39m) at complex.jl:254
  *(::Real, [91m::Complex[39m) at complex.jl:266
  ...[39m

Let's see what happened if we define a function without a catch-all version.

In [7]:
echo(s::String) = (s * "!")^3

echo (generic function with 1 method)

In [8]:
@show echo("slam")
@show echo("8")
@show echo(:slam)

echo("slam") = "slam!slam!slam!"
echo("8") = "8!8!8!"


LoadError: [91mMethodError: no method matching echo(::Symbol)[0m
Closest candidates are:
  echo([91m::String[39m) at In[7]:1[39m

---

## Have C and Python binding

Interfacing with shared libraries written in C is straightforward. For example, say we have a `libhello.so` created from [hello.c](hello.c).

In [9]:
ccall((:hello, "libhello.so"), Void, (Cstring,), "Moto")

Hello, Moto!


Calling Julia functions from C can be done by including `julia.h` that can be found inside the Julia installation (hereby handwaved as it's outside the scope of this sharing).

Julia can call Python objects using the `PyCall.jl` library.

In [10]:
using PyCall

In [11]:
py"map(lambda x: 2 * x, [1, 2, 3, 4])"

4-element Array{Int64,1}:
 2
 4
 6
 8

In [12]:
@pycall PyObject(map)(x -> 2x, [1, 2, 3, 4])::Vector{Int}

4-element Array{Int64,1}:
 2
 4
 6
 8

Calling Julia functions from Python can be done using the Python library `PyJulia` (hereby handwaved again as it's outside the scope of this sharing).

---

## Can pop open the hood

Methods introspection - See intermediate and compiled representations.

In [13]:
methods(double)

# 2 methods for generic function "double":
double(s::Array{T,1} where T) in Main at In[5]:1
double(x) in Main at In[4]:2

In [14]:
@code_typed double(5)

CodeInfo(:(begin 
        return (Base.mul_int)(2, x)::Int64
    end))=>Int64

In [15]:
@code_typed double(5.0)

CodeInfo(:(begin 
        return (Base.mul_float)((Base.sitofp)(Float64, 2)::Float64, x)::Float64
    end))=>Float64

In [16]:
@code_typed double([4, 5, 6])

CodeInfo(:(begin 
        return $(Expr(:invoke, MethodInstance for vcat(::Array{Int64,1}, ::Array{Int64,1}), :(Main.vcat), :(s), :(s)))
    end))=>Array{Int64,1}

In [17]:
@code_native double(5)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[4]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	leaq	(%rdi,%rdi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


In [18]:
@code_native double(5.0)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[4]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	addsd	%xmm0, %xmm0
	popq	%rbp
	retq
	nopw	(%rax,%rax)


Constant function calls in a function definition are run during compilation for speedups.

In [19]:
what(x) = double(5) + double(6) + x

what (generic function with 1 method)

In [20]:
@code_native what(5566)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[19]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	leaq	22(%rdi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


Example: Two ways to write the same function...

In [21]:
function task_1()
    result = 0
    for i in 1:100000
        result += 1/(i^2)
    end
    return result
end

task_1 (generic function with 1 method)

In [22]:
function task_2()
    result = 0.0
    for i in 1:100000
        result += 1/(i^2)
    end
    return result
end

task_2 (generic function with 1 method)

Let's time it

In [23]:
task_1() #First time is to compile the method once.
@time task_1()

  0.006320 seconds (300.08 k allocations: 4.584 MiB)


1.6449240668982423

In [24]:
task_2() #First time is to compile the method once.
@time task_2()

  0.000415 seconds (5 allocations: 176 bytes)


1.6449240668982423

What happened?

In [25]:
@code_warntype task_1()

Variables:
  #self# <optimized out>
  i::Int64
  #temp#@_3::Int64
  result::Union{Float64, Int64}
  #temp#@_5::Core.MethodInstance
  #temp#@_6::Float64

Body:
  begin 
      result::Union{Float64, Int64} = 0 # line 3:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1, 100000)::Bool, 100000, (Base.sub_int)(1, 1)::Int64)::Int64
      #temp#@_3::Int64 = 1
      5: 
      unless (Base.not_int)((#temp#@_3::Int64 === (Base.add_int)(SSAValue(2), 1)::Int64)::Bool)::Bool goto 30
      SSAValue(3) = #temp#@_3::Int64
      SSAValue(4) = (Base.add_int)(#temp#@_3::Int64, 1)::Int64
      i::Int64 = SSAValue(3)
      #temp#@_3::Int64 = SSAValue(4) # line 4:
      unless (result::Union{Float64, Int64} isa Int64)::Bool goto 15
      #temp#@_5::Core.MethodInstance = MethodInstance for +(::Int64, ::Float64)
      goto 24
      15: 
      unless (result::Union{Float64, Int64} isa Float64)::Bool goto 19
      #temp#@_5::Core.MethodInstance = MethodInstance for +(::Float64, ::Float64)
      goto 24
 

In [26]:
@code_warntype task_2()

Variables:
  #self# <optimized out>
  i::Int64
  #temp#::Int64
  result::Float64

Body:
  begin 
      result::Float64 = 0.0 # line 3:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1, 100000)::Bool, 100000, (Base.sub_int)(1, 1)::Int64)::Int64
      #temp#::Int64 = 1
      5: 
      unless (Base.not_int)((#temp#::Int64 === (Base.add_int)(SSAValue(2), 1)::Int64)::Bool)::Bool goto 15
      SSAValue(3) = #temp#::Int64
      SSAValue(4) = (Base.add_int)(#temp#::Int64, 1)::Int64
      i::Int64 = SSAValue(3)
      #temp#::Int64 = SSAValue(4) # line 4:
      result::Float64 = (Base.add_float)(result::Float64, (Base.div_float)((Base.sitofp)(Float64, 1)::Float64, (Base.sitofp)(Float64, (Base.mul_int)(i::Int64, i::Int64)::Int64)::Float64)::Float64)::Float64
      13: 
      goto 5
      15:  # line 6:
      return result::Float64
  end::Float64


Type introspection - work with the relationship between types.

In [27]:
typeof(5)

Int64

In [28]:
Int64 <: Integer

true

In [29]:
is_an_integer(x::Integer) = true
is_an_integer(x) = false

is_an_integer (generic function with 2 methods)

In [30]:
@show is_an_integer(5)
@show is_an_integer(5.0)

is_an_integer(5) = true
is_an_integer(5.0) = false


false

Strong reflection makes it easy and fast to implement much of Julia in Julia.

In [31]:
is_type{T}(x::T, t::Type{T}) = true
is_type(x, t) = false

is_type (generic function with 2 methods)

In [32]:
@show is_type(5.0, Float64)
@show is_type(5, Int)
@show is_type(5, String)
@show is_type(is_type, Function)

is_type(5.0, Float64) = true
is_type(5, Int) = true
is_type(5, String) = false
is_type(is_type, Function) = true


true

Reflection

In [33]:
@show double(6)
@show double(7.5)

double(6) = 12
double(7.5) = 15.0


15.0

In [34]:
input(text::String) = (print("$(text): "); return readline())

function run_function_on_demand()
    f = Symbol(input("Function to run"))
    v = input("what's the numerical value")
    t = Symbol(input("what's the value's type"))
    eval(:($f(parse($t, $v))))
end

run_function_on_demand()

Function to run: STDIN> double
what's the numerical value: STDIN> 6
what's the value's type: STDIN> Int64


12

---

## Malleable and expressive

Here's an integer that keep track of it's derivation, otherwise known as Big Shaq Integer (`BSInt`).

In [35]:
struct BSInt
    value::Int
    history::String
end

In [36]:
BSInt(value::Int) = BSInt(value, "$value")

Base.:+(x::BSInt, y::BSInt) = Base.:+(x, y.value)
function Base.:+(x::BSInt, y::Int)
    value = x.value + y
    history = "$(x.history) plus $y is $value"
    return BSInt(value, history)
end

Base.:-(x::BSInt, y::BSInt) = Base.:-(x, y.value)
function Base.:-(x::BSInt, y::Int)
    value = x.value - y
    history = "$(x.history) minus $y that's $value"
    return BSInt(value, history)
end

Base.show(io::IO, x::BSInt) = show(io, "$(x.history), quick maffs.")

In [37]:
two = BSInt(2)
two + two - 1

"2 plus 2 is 4 minus 1 that's 3, quick maffs."

In [38]:
v = [2, 4, 6, 8, 10]
@show v[1]
@show v[2]
v[2] = 100
@show v;

v[1] = 2
v[2] = 4
v = [2, 100, 6, 8, 10]


Don't like Julia's 1-indexing? Create and use your own vector.

In [39]:
struct MyVector{T}
    data::Vector{T}
end
function Base.setindex!(v::MyVector, x, i)
    v.data[i + 1] = x
end

Base.getindex(v::MyVector, i) = v.data[i + 1]

In [40]:
v = MyVector([2, 4, 6, 8, 10])

MyVector{Int64}([2, 4, 6, 8, 10])

In [41]:
@show v[1]
@show v[2]
v[2] = 100
@show v;

v[1] = 4
v[2] = 6
v = MyVector{Int64}([2, 4, 100, 8, 10])


Don't like for-loops from the front? Do from the back!

In [42]:
Base.start(v::MyVector) = length(v.data) - 1
Base.done(v::MyVector, state) = state < 0
Base.next(v::MyVector, state) = (v[state], state - 1)

In [43]:
for elem in v
    println(elem)
end

10
8
100
4
2


In [44]:
sum(v)

124

Is it possible to return `true` for the following expression?
```julia
(1 == a) && (2 == a) && (3 == a)
```

In [45]:
struct Ditto end
Base.:(==)(lhs, ::Ditto) = true
a = Ditto()

Ditto()

In [46]:
(1 == a) && (2 == a) && (3 == a)

true

In [47]:
mutable struct UpInt
    value::Int
end
function Base.:(==)(lhs, i::UpInt)
    i.value += 1
    return lhs == i.value
end
a = UpInt(0)

UpInt(0)

In [48]:
(1 == a) && (2 == a) && (3 == a)

true

---

## Supports Unicode operators

In [49]:
4 / 3

1.3333333333333333

In [50]:
4 // 3

4//3

In [51]:
div(4, 3)

1

In [52]:
4 ÷ 3

1

Operators are not special constructs. They are just syntactic sugar for the corresponding function.

In [53]:
÷(4, 3)

1

In [54]:
Base.:÷(4, 3)

1

In [55]:
typeof(:÷)

Symbol

In [56]:
π

π = 3.1415926535897...

In [57]:
α₁ = 3.0; α₂ = 4.0
√(α₁^2 + α₂^2)

5.0

In [58]:
fib(n) = (n ≤ 1) ? n : fib(n - 1) + fib(n - 2)

fib (generic function with 1 method)

In [59]:
@time fib(40)
@time fib(40)
@time fib(40)

  0.717460 seconds (505 allocations: 28.762 KiB)
  0.703402 seconds (5 allocations: 176 bytes)
  0.691830 seconds (5 allocations: 176 bytes)


102334155

---

## Good for data processing

Julia has Vector that are specialised to the type, implemented as efficiently as possible.

In [60]:
like_a_c_array = [5, 5, 6, 6]

4-element Array{Int64,1}:
 5
 5
 6
 6

In [61]:
a = [1, "is", sum, "number"]

4-element Array{Any,1}:
 1        
  "is"    
  sum     
  "number"

In [62]:
foods_i_like = ["Pasta", "茶そば", "Chicken Pea", "Starbucks"]

4-element Array{String,1}:
 "Pasta"      
 "茶そば"        
 "Chicken Pea"
 "Starbucks"  

Vector is just a 1-D array (`Vector{T}` is alias for `Array{T, 1}`). Julia talks naturally in n-D.

In [63]:
hcat(like_a_c_array, a, foods_i_like)

4×3 Array{Any,2}:
 5  1          "Pasta"      
 5   "is"      "茶そば"        
 6   sum       "Chicken Pea"
 6   "number"  "Starbucks"  

Common to compiled languages, there is quick for-loop, and so there is not a need to vectorise for performance.

In [64]:
for food in foods_i_like
    println("I like $(lowercase(food)) oh yeah~~")
end

I like pasta oh yeah~~
I like 茶そば oh yeah~~
I like chicken pea oh yeah~~
I like starbucks oh yeah~~


Common to interpreted languages, functions are first class object. So we have a powerful `do` construct.

In [65]:
foreach(foods_i_like) do food
    println("I like $(lowercase(food)) oh yeah~~")
end

I like pasta oh yeah~~
I like 茶そば oh yeah~~
I like chicken pea oh yeah~~
I like starbucks oh yeah~~


In [66]:
foreach(food -> println("I like $(lowercase(food)) oh yeah~~"), foods_i_like)

I like pasta oh yeah~~
I like 茶そば oh yeah~~
I like chicken pea oh yeah~~
I like starbucks oh yeah~~


Many representation of common data processing constructs.

In [67]:
foods = ["Prata", "油条", "Chicken Rice", "Starbucks"];

In [68]:
map(food -> food ∈ foods_i_like, foods)

4-element Array{Bool,1}:
 false
 false
 false
  true

In [69]:
filter(food -> food ∈ foods_i_like, foods)

1-element Array{String,1}:
 "Starbucks"

In [70]:
map(foods) do food
    return food ∈ foods_i_like
end

4-element Array{Bool,1}:
 false
 false
 false
  true

In [71]:
filter(foods) do food
    return food ∈ foods_i_like
end

1-element Array{String,1}:
 "Starbucks"

In [72]:
[food ∈ foods_i_like for food in foods]

4-element Array{Bool,1}:
 false
 false
 false
  true

In [73]:
[food for food in foods if food ∈ foods_i_like]

1-element Array{String,1}:
 "Starbucks"

A reason to do vectorisation is for code clarity at the expense of speed. Julia has a speedy way (both to code and to run) to do that, using syntactic loop fusion.

Say $f(x) = 3x^2 + 5x + 2$, we want to compute $f(2x^2 + 6x^3 - \sqrt{x})$ for all $x \in X$.

In [74]:
f(x) = 3x^2 + 5x + 2

f (generic function with 1 method)

In [75]:
X = collect(0.001:0.002:1.0)
function manual_map!(X)
    for i in eachindex(X)
        X[i] = f(2X[i]^2 + 6X[i]^3 - sqrt(X[i]))
    end
end
manual_map!(X) # First run to compile the function.
@time manual_map!(X)
sum(1 ./ X)

  0.000005 seconds (4 allocations: 160 bytes)


2.7665013801459013

In [76]:
X = collect(0.001:0.002:1.0)
function vec_map_1!(X)
    X .= f.(2 .* X.^2 .+ 6 .* X.^3 .- sqrt.(X))
    return
end
vec_map_1!(X) # First run to compile the function.
@time vec_map_1!(X)
sum(1 ./ X)

  0.000004 seconds (4 allocations: 160 bytes)


2.7665013801459013

In [77]:
X = collect(0.001:0.002:1.0)
function vec_map_2!(X)
    @. X = f(2X^2 + 6X^3 - sqrt(X))
    return
end
vec_map_2!(X) # First run to compile the function.
@time vec_map_2!(X)
sum(1 ./ X)

  0.000005 seconds (4 allocations: 160 bytes)


2.7665013801459013

In [78]:
X = collect(0.001:0.002:1.0)
h(x) = 2x^2 + 6x^3 - √x
function vec_map_3!(X)
    X .= ((f ∘ h).(X))
    return
end
vec_map_3!(X) # First run to compile the function.
@time vec_map_3!(X)
sum(1 ./ X)

  0.000005 seconds (4 allocations: 160 bytes)


2.7665013801459013

Plotting is straightforward using [PLots.jl](https://github.com/JuliaPlots/Plots.jl). We can choose which plotting backend (e.g. matplotlib, GR, PlotlyJS) to use due to Julia's reflection.

In [79]:
data = readcsv("ex1data1.txt");

In [80]:
using Plots
gr() # Use the GR plotting library instead

function plot_data(X, y)
    scatter(X, y, marker = (:cross, :red))
end

plot_data (generic function with 1 method)

In [81]:
X = data[:, 1]; y = data[:, 2];
m = length(y);
plot_data(X, y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

We can easily convert mathematical equations into code, which is a daily challenge for data scientist.

Say, we have $y \in \mathbb{R}_m$, $X \in \mathbb{R}_{m, n}$

Firstly, prepend the intercept column to $X$, i.e. $X \rightarrow [1_m \ X]$

Cost function: $J_{X, y}(\theta) = \dfrac{1}{2m} (X \theta - y)^T (X \theta - y) = \dfrac{1}{2m} (X \theta - y) \cdot (X \theta - y)$

Cost gradient: $\partial J_{X, y} (\theta) = \dfrac{1}{m} X^T (X \theta - y)$

For a given $\alpha$ and $\theta$, run $\theta \rightarrow \theta - \alpha \ \partial J_{X, y}(\theta) = \theta - \dfrac{\alpha}{m} X^T (X \theta - y)$ for given number of iterations to approach optimal $\theta$.

In [82]:
prepend_intercept(A) = [ones(eltype(A), size(A, 1)) A]

prepend_intercept (generic function with 1 method)

In [83]:
function make_cost_functions(X, y)
    m = length(y)
    
    function J(θ)
        δ = X * θ .- y
        return (δ ⋅ δ) / (2m)
    end
    
    function ∂J(θ)
        δ = X * θ .- y
        return (X' * δ) / m
    end
    return J, ∂J
end

make_cost_functions (generic function with 1 method)

In [84]:
function run_gradient_descent(X, y, θ, α, iterations)
    X = prepend_intercept(X)

    J, ∂J = make_cost_functions(X, y)
    
    println("Initial: θ = $θ; Cost == $(J(θ))")
    for i in 1:iterations
        θ = θ - α * ∂J(θ)
        (i % 100 == 0) && println("Iteration $i: θ = $θ; Cost == $(J(θ)).")
    end
    
    return θ
end

run_gradient_descent (generic function with 1 method)

In [85]:
α = 0.01
θ = [0, 0]
iterations = 1500
θ = run_gradient_descent(X, y, θ, α, iterations)

@show θ; # Expected θ ≈ [-3.6303, 1.1664]

Initial: θ = [0, 0]; Cost == 32.072733877455676
Iteration 100: θ = [-0.576556, 0.859582]; Cost == 5.479975580535112.
Iteration 200: θ = [-1.1245, 0.914629]; Cost == 5.176152962891967.
Iteration 300: θ = [-1.58199, 0.960588]; Cost == 4.964362046184744.
Iteration 400: θ = [-1.96396, 0.998961]; Cost == 4.816725268028978.
Iteration 500: θ = [-2.28287, 1.031]; Cost == 4.713809531116866.
Iteration 600: θ = [-2.54913, 1.05775]; Cost == 4.642068268292856.
Iteration 700: θ = [-2.77144, 1.08008]; Cost == 4.592058338363686.
Iteration 800: θ = [-2.95705, 1.09873]; Cost == 4.557197046645756.
Iteration 900: θ = [-3.11202, 1.1143]; Cost == 4.532895679657961.
Iteration 1000: θ = [-3.2414, 1.12729]; Cost == 4.515955503078913.
Iteration 1100: θ = [-3.34943, 1.13815]; Cost == 4.504146719537086.
Iteration 1200: θ = [-3.43962, 1.14721]; Cost == 4.495914965394088.
Iteration 1300: θ = [-3.51493, 1.15477]; Cost == 4.490176713352229.
Iteration 1400: θ = [-3.5778, 1.16109]; Cost == 4.48617665019146.
Iteration 1

Ending this section, a new v0.7 feature that I find interesting and further simplify representation for data science.

In [ ]:
# Coming in v0.7
distance((x₁, y₁), (x₂, y₂)) = √((y₂ - y₁)^2 + (x₂, x₁)^2)

distance((0, 3), (4, 0)) # will returns 5.0

In [86]:
typeof((1, 2, "3"))

Tuple{Int64,Int64,String}

---

## Meta-programming

Like Lisp, we can write macros like functions, which takes in S-expressions as arguments instead.

In [87]:
ex = :(2 + 3 == 4 + 1)

:(2 + 3 == 4 + 1)

In [88]:
@show typeof(ex) ex.head ex.args;

typeof(ex) = Expr
ex.head = :call
ex.args = Any[:(==), :(2 + 3), :(4 + 1)]


The syntax for creating our own macro is straightforward.

In [89]:
macro myassert(expr)
    if typeof(expr) == Expr && expr.head == :call && expr.args[1] == :(==)
        return :(println("$($(string(expr))) is $($(expr))"))
    else
        return :(println("mai la"))
    end
end

@myassert (macro with 1 method)

In [90]:
@myassert 2 + 3 == 1 + 4

2 + 3 == 1 + 4 is true


In [91]:
@myassert "I luv prata"

mai la


In [92]:
@myassert 4 < 5

mai la


---

## Straightforward concurrency and parallelism

What's better to show existence of concurrency than sleep sort?

In [93]:
array = [2, 4, 3, 2, 3, 1, 1, 4];

In [94]:
c = Channel{Int}(0)

for value in array
    @schedule (sleep(value * 0.1); put!(c, value))
end

[take!(c) for _ in array]

8-element Array{Int64,1}:
 1
 1
 2
 2
 3
 3
 4
 4

Also for the bigger parallelizable problems there is true parallelization.

In [95]:
@show nprocs() nworkers();
addprocs(2)
@show nprocs() nworkers();

nprocs() = 1
nworkers() = 1
nprocs() = 3
nworkers() = 2


In [96]:
@everywhere using Primes

In [97]:
sum_prime_1() = sum(prime(i) for i in 1:3000)

sum_prime_1()
@time sum_prime_1()

  4.256737 seconds (5.92 M allocations: 90.339 MiB, 0.20% gc time)


38645211

In [98]:
function sum_prime_2()
    @parallel (+) for i in 1:3000
        prime(i)
    end
end

sum_prime_2()
@time sum_prime_2()

  3.406933 seconds (332 allocations: 20.109 KiB)


38645211

---

## Demo

Did this little program one afternoon by the pool brogramming with wife =P

It converts between various bitstrings of various bitsizes as fast as possible for crypto use. To achieve that, I will have to write many almost-repetitive functions (if I have $n$ bitsizes, I will write $O(n^2)$ functions).

Compile languages aims to be slim and seldom have the reflection required to reduce written functions. Dynamic languages tend to lack type consciousness to generate efficient functions from the reflection done.

Let's see how Julia deals with the struggles.

In [99]:
Word = Union{UInt8, UInt16, UInt32, UInt64}

Union{UInt16, UInt32, UInt64, UInt8}

I want an easy way to read in a word(which is a hexadecimal strings). It will be nice if I can do something as simple as

```
value = 0x0123456789abcdef0011223344556677
# cannot read in directly if system is not at least 128-bit
```

compared to the natural way to read

```
value = [0x01, 0x23, 0x45, 0x67,
         0x89, 0xab, 0xcd, 0xef,
         0x00, 0x11, 0x22, 0x33,
         0x44, 0x55, 0x66, 0x77]
```

so that the crypto test vectors (usually 256-bit, 512-bit, or even longer) can be easily read in code.

In [100]:
hex2bytes("0123456789abcdef0011223344556677")

16-element Array{UInt8,1}:
 0x01
 0x23
 0x45
 0x67
 0x89
 0xab
 0xcd
 0xef
 0x00
 0x11
 0x22
 0x33
 0x44
 0x55
 0x66
 0x77

In [101]:
macro x_str(str)
    return hex2bytes(str)
end

@x_str (macro with 1 method)

In [102]:
x"0123456789abcdef0011223344556677"

16-element Array{UInt8,1}:
 0x01
 0x23
 0x45
 0x67
 0x89
 0xab
 0xcd
 0xef
 0x00
 0x11
 0x22
 0x33
 0x44
 0x55
 0x66
 0x77

In [103]:
num_bits(::Type{UInt8}) = 8
num_bits(::Type{UInt16}) = 16
num_bits(::Type{UInt32}) = 32
num_bits(::Type{UInt64}) = 64

num_bits (generic function with 4 methods)

In [104]:
num_bytes{T <: Word}(::Type{T}) = num_bits(T) ÷ 8

mask{T <: Word}(::Type{T}) = -one(T)

mask (generic function with 1 method)

In [105]:
function Base.convert(::Type{Vector{T1}}, v::Vector{T2}) where T1 <: Word where T2 <: Word
    if num_bytes(T1) > num_bytes(T2)
        return pack(Vector{T1}, v)
    elseif num_bytes(T1) < num_bytes(T2)
        return unpack(Vector{T1}, v)
    else
        return v
    end
end


function pack(::Type{Vector{T1}}, v::Vector{T2}) where T1 <: Word where T2 <: Word
    width_result = num_bytes(T1) ÷ num_bytes(T2)
    length_result = length(v) ÷ width_result

    result = zeros(T1, length_result)

    k = 1
    for i in 1:length_result
        value = zero(T1)
        for j in 1:width_result
            value <<= num_bits(T2)
            value |= v[k]
            k += 1
        end
        result[i] = value
    end

    return result
end

function unpack(::Type{Vector{T1}}, v::Vector{T2}) where T1 <: Word where T2 <: Word
    width_source = num_bytes(T2) ÷ num_bytes(T1)
    length_result = length(v) * width_source

    result = zeros(T1, length_result)

    i = length_result
    k = length(v)
    while i > 0
        value = v[k]
        for j in 1:width_source
            result[i] = value & mask(T1)
            value >>= num_bits(T1)
            i -= 1
        end
        k -= 1
    end

    return result
end

unpack (generic function with 1 method)

In [106]:
s = convert(Vector{UInt32}, x"0123456789abcdef0011223344556677")

4-element Array{UInt32,1}:
 0x01234567
 0x89abcdef
 0x00112233
 0x44556677

In [107]:
convert(Vector{UInt8}, s)

16-element Array{UInt8,1}:
 0x01
 0x23
 0x45
 0x67
 0x89
 0xab
 0xcd
 0xef
 0x00
 0x11
 0x22
 0x33
 0x44
 0x55
 0x66
 0x77

In [108]:
@code_llvm convert(Vector{UInt8}, x"0123456789abcdef")


define i8** @julia_convert_65919(i8**, i8** dereferenceable(40)) #0 !dbg !5 {
L13:
  ret i8** %1
}


In [109]:
@code_llvm convert(Vector{UInt32}, x"0123456789abcdef")


define i8** @julia_convert_65863(i8**, i8** dereferenceable(40)) #0 !dbg !5 {
if:
  %2 = call i8** @julia_pack_65864(i8** inttoptr (i64 4436047088 to i8**), i8** nonnull %1)
  ret i8** %2
}


---

<div style="font-size:48px; text-align: center">
Thank you! =)
</div>